# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Цель исследования**   
Проанализировать базу данных (информация о книгах, издательствах, авторах, пользовательские обзоры книг), чтобы сформулировать ценностное предложение для нового продукта.

**Описание данных**  

Таблица `books` — содержит данные о книгах:

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.

Таблица `authors` — содержит данные об авторах:

    author_id — идентификатор автора;
    author — имя автора.

Таблица `publishers` — содержит данные об издательствах:

    publisher_id — идентификатор издательства;
    publisher — название издательства;

Таблица `ratings` — содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.

Таблица `reviews` — содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры
db_config = {'user': '_______', # имя пользователя
'pwd': '_________', # пароль
'host': '________', 
'port': ______, # порт подключения
'db': '_________'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# напишем функцию для выполнения запроса
def select(query):
    return pd.io.sql.read_sql(query, con = engine)

## Исследование таблиц

In [3]:
def get_info(df):
    print('------ Первые 5 строк ------')
    display(df.head())
    print()
    print('------ Типы данных ------')
    df.info()
    print()
    display(df.describe())
    display(df.describe(include='object'))
    print()
    print('------ Уникальные значения в файле ------')
    display(df.nunique())
    print()
    print('------ Пропуски в файле ------')
    display(df.isna().sum())
    print()
    print('------ Дубликаты в файле ------')
    display(df.duplicated().sum())

### Таблица `books`

In [4]:
get_info(
    select(
        '''
            SELECT *
            FROM books
        '''
    )
)

------ Первые 5 строк ------


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



------ Типы данных ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB



,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000


,title,publication_date
count,1000,1000
unique,999,618
top,Memoirs of a Geisha,2006-08-29
freq,2,10


...

book_id             1000
author_id            636
title                999
num_pages            454
publication_date     618
publisher_id         340
dtype: int64


------ Пропуски в файле ------


book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64


------ Дубликаты в файле ------


0

Пропуски и дубликаты отсутствуют. Название книги `Memoirs of a Geisha` встречается 2 раза.

In [5]:
# найдем книгу Memoirs of a Geisha
books = select(
        '''
            SELECT *
            FROM books
        '''
    )

books[books['title'] == "Memoirs of a Geisha"]

,book_id,author_id,title,num_pages,publication_date,publisher_id
425,426,39,Memoirs of a Geisha,434,2005-11-15,241
426,427,39,Memoirs of a Geisha,503,2005-11-22,311


Данную книгу одного автора выпускали два издательства с разным количеством страниц и под разным номером id книги.

### Таблица `authors`

In [6]:
get_info(
    select(
        '''
            SELECT *
            FROM authors
        '''
    )
)

------ Первые 5 строк ------


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



------ Типы данных ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB



,author_id
count,636.000000
mean,318.500000
std,183.741666
min,1.000000
25%,159.750000
50%,318.500000
75%,477.250000
max,636.000000


,author
count,636
unique,636
top,Clayton M. Christensen/L.J. Ganser
freq,1


...

author_id    636
author       636
dtype: int64


------ Пропуски в файле ------


author_id    0
author       0
dtype: int64


------ Дубликаты в файле ------


0

Пропуски и дубликаты отсутствуют.

### Таблица `publishers`

In [7]:
get_info(
    select(
        '''
            SELECT *
            FROM publishers
        '''
    )
)

------ Первые 5 строк ------


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



------ Типы данных ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB



,publisher_id
count,340.000000
mean,170.500000
std,98.293777
min,1.000000
25%,85.750000
50%,170.500000
75%,255.250000
max,340.000000


,publisher
count,340
unique,340
top,HarperCollins Publishers
freq,1


...

publisher_id    340
publisher       340
dtype: int64


------ Пропуски в файле ------


publisher_id    0
publisher       0
dtype: int64


------ Дубликаты в файле ------


0

Пропуски и дубликаты отсутствуют.

In [8]:
# найдем издателей книги Memoirs of a Geisha
publishers = select(
        '''
            SELECT *
            FROM publishers
        '''
    )
publishers.query('publisher_id == 241 or publisher_id == 311')

,publisher_id,publisher
240,241,Random House Large Print Publishing
310,311,Vintage Books USA


Книга `Memoirs of a Geisha` было выпущено двумя издателями: `Random House Large Print Publishing` и `Vintage Books USA`.

### Таблица `ratings`

In [9]:
get_info(
    select(
        '''
            SELECT *
            FROM ratings
        '''
    )
)

------ Первые 5 строк ------


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



------ Типы данных ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB



,rating_id,book_id,rating
count,6456.000000,6456.000000,6456.000000
mean,3228.500000,510.574195,3.928284
std,1863.831001,284.141636,0.943303
min,1.000000,1.000000,1.000000
25%,1614.750000,291.000000,3.000000
50%,3228.500000,506.000000,4.000000
75%,4842.250000,750.000000,5.000000
max,6456.000000,1000.000000,5.000000


,username
count,6456
unique,160
top,paul88
freq,56


...

rating_id    6456
book_id      1000
username      160
rating          5
dtype: int64


------ Пропуски в файле ------


rating_id    0
book_id      0
username     0
rating       0
dtype: int64


------ Дубликаты в файле ------


0

Пропуски и дубликаты отсутствуют.  
Наиболее активным оценщиком является пользователь под ником paul88, который 56 раз ставил оценки.  
Всего уникальных пользователей 160.  
Оценка осуществляется по 5-ти бальной системе.

### Таблица `reviews`

In [10]:
get_info(
    select(
        '''
            SELECT *
            FROM reviews
        '''
    )
)

------ Первые 5 строк ------


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



------ Типы данных ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB



,review_id,book_id
count,2793.000000,2793.000000
mean,1397.000000,504.693161
std,806.413976,288.472931
min,1.000000,1.000000
25%,699.000000,259.000000
50%,1397.000000,505.000000
75%,2095.000000,753.000000
max,2793.000000,1000.000000


,username,text
count,2793,2793
unique,160,2793
top,susan85,Language win analysis research debate life the...
freq,29,1


...

review_id    2793
book_id       994
username      160
text         2793
dtype: int64


------ Пропуски в файле ------


review_id    0
book_id      0
username     0
text         0
dtype: int64


------ Дубликаты в файле ------


0

Пропуски и дубликаты отсутствуют.  
Наиболее активным ревьюером является пользователь под ником susan85, который 29 раз оставляла обзоры.  
Всего уникальных пользователей 160.

## Задания

### Количество книг, вышедших после 1 января 2000 года

In [12]:
select(
    '''
        SELECT COUNT(book_id) AS books_count
        FROM books
        WHERE publication_date > '01-01-2000'
    '''
)

,books_count
0,819


После 1 января 2000 года всего было выпущено 819 книг.

### Количество обзоров и средняя оценка каждой книги

In [13]:
select(
    '''
        SELECT b.book_id,
            b.title,
            COUNT(DISTINCT reviews.review_id) AS reviews_count,
            ROUND(AVG(ratings.rating),2) AS avg_rating
        FROM books AS b
        FULL JOIN reviews ON reviews.book_id = b.book_id
        FULL JOIN ratings ON ratings.book_id = b.book_id
        GROUP BY b.book_id
        ORDER BY reviews_count DESC,
                avg_rating DESC
        LIMIT 5
    '''
)

,book_id,title,reviews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21


Количество обзоров на книги варьируется от 0 до 7, рейтинг от 1.50 до 5.00:
- максимальное количество обзоров у книги - Twilight (Twilight #1) - 7;
- минимальный средний рейтинг у книги Harvesting the Heart - 1.50.


### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [14]:
select(
    '''
        SELECT p.publisher,
            COUNT(books.book_id) AS books_count
        FROM publishers AS p
        FULL JOIN books ON books.publisher_id = p.publisher_id
        WHERE books.num_pages > 50
        GROUP BY p.publisher
        ORDER BY books_count DESC
        LIMIT 5
    '''
)

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Нибольшее число книг толще 50 страниц выпустило издательство Penguin Books, всего - 42 книги.

### Автор с самой высокой средней оценкой книг (книги с 50 и более оценками)

In [15]:
select(
    '''
        SELECT i.author,
            ROUND(AVG(i.avg_rating),2) AS avg_rating
        FROM 
            (SELECT authors.author,
                books.book_id,
                AVG(ratings.rating) AS avg_rating
             FROM authors
             FULL JOIN books ON books.author_id = authors.author_id
             FULL JOIN ratings ON ratings.book_id = books.book_id
             GROUP BY author,
                 books.book_id
             HAVING COUNT (ratings.rating) >= 50
            ) AS i
        GROUP BY author
        ORDER BY avg_rating DESC
        LIMIT 5
    '''
)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


Автором с самой высокой средней оценкой книг (книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré, средняя оценка - 4,28.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [16]:
select(
    '''
        SELECT ROUND(AVG(i.reviews_count)) AS avg_reviews
        FROM 
            (SELECT COUNT(reviews.review_id) AS reviews_count
             FROM reviews
             WHERE username IN
                 (SELECT username
                  FROM ratings
                  GROUP BY username
                  HAVING COUNT(rating_id) > 50
                  )
             GROUP BY username
            ) AS i
    '''
)

,avg_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24.

## Общее заключение

- После 1 января 2000 года всего было выпущено 819 книг;
- Количество обзоров на книги варьируется от 0 до 7, рейтинг от 1.50 до 5.00;
- Максимальное количество обзоров у книги - Twilight (Twilight #1) - 7;
- Минимальный средний рейтинг у книги Harvesting the Heart - 1.50;
- Нибольшее число книг выпустило издательство Penguin Books, всего - 42 книги;
- Автором с самой высокой средней оценкой книг (книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré, средняя оценка - 4,28;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24.